# Ukraine food graphic
*June 8, 2022*

The news is very focused on the war in Ukraine's impact on world food supplies. There's some data over at [OWID](https://ourworldindata.org/explorers/ukraine-russia-food?tab=table&facet=none&Crop=Sunflower+oil&Metric=Imports&Adjustment=Share+of+imports&Imports+from...=Ukraine&country=EGY~IDN~MAR~TUN~ALB~DZA~FJI) that we can use to create a graphic that helps show how dependent the world is on Ukraine's crops.

First, we import pandas and a library called datawrappergraphics (that I made) that will help us send the data to Datawrapper to visualize.

In [26]:
import pandas as pd
import datawrappergraphics
import logging

logging.getLogger().setLevel(logging.CRITICAL)

Now we import the raw data and take a quick peek.

In [27]:
raw = pd.read_csv("ukraine-russia-food.csv")

raw.head(3)

,Domestic wheat supply,Wheat exports,Wheat imports,Wheat stocks,Net wheat imports,Wheat imports (% domestic supply),Wheat stocks (% domestic supply),Wheat exports per capita,Wheat imports per capita,Net wheat imports per capita,...,Sunflower oil imports from Russia per capita,Sunflower oil imports from Ukraine + Russia per capita,Sunflower oil imports from Ukraine (% imports),Sunflower oil imports from Russia (% imports),Sunflower oil imports from Ukraine + Russia (% imports),Sunflower oil imports from Ukraine (% supply),Sunflower oil imports from Russia (% supply),Sunflower oil imports from Ukraine + Russia (% supply),Entity,Year
0,6209000,NaN,206145.0,182000,NaN,NaN,2.931229,NaN,7.063265,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,2010
1,6011000,NaN,220428.0,-402000,NaN,NaN,-6.687739,NaN,7.318956,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,2011
2,6186000,NaN,204801.0,497000,NaN,NaN,8.034271,NaN,6.572270,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,2012


Now we filter for just the columns we want: the location, the year, and anything that shows % of imports coming from Ukraine (ie. not Russia).

In [28]:
data = raw.loc[:, raw.columns.str.contains("Entity") | raw.columns.str.contains("Year") | (raw.columns.str.contains("% import") & ~raw.columns.str.contains("Russia"))]

data.tail(3)

,Wheat imports from Ukraine (% imports),Maize imports from Ukraine (% imports),Barley imports from Ukraine (% imports),Sunflower oil imports from Ukraine (% imports),Entity,Year
1841,NaN,NaN,NaN,NaN,Zimbabwe,2017
1842,2.312339,NaN,NaN,NaN,Zimbabwe,2018
1843,18.176712,NaN,NaN,NaN,Zimbabwe,2019


We could now filter for only entries for the most recent year (2019), but what if countries don't have data for 2019? We want to include their latest report here. So instead, we get the most recent year for each country.

In [29]:
data = (data
        .dropna(subset=data.columns[data.columns.str.contains("imports")], how="all")
        .sort_values(["Entity", "Year"], ascending=False)
        .drop_duplicates("Entity")
        )

data.head(3)

,Wheat imports from Ukraine (% imports),Maize imports from Ukraine (% imports),Barley imports from Ukraine (% imports),Sunflower oil imports from Ukraine (% imports),Entity,Year
1843,18.176712,NaN,NaN,NaN,Zimbabwe,2019
1829,19.770719,NaN,NaN,NaN,Zambia,2015
1823,11.933531,NaN,NaN,4.896368,Yemen,2019


In [30]:
data[data["Entity"] == "Libya"]

,Wheat imports from Ukraine (% imports),Maize imports from Ukraine (% imports),Barley imports from Ukraine (% imports),Sunflower oil imports from Ukraine (% imports),Entity,Year
954,43.341057,70.219635,33.741733,16.885063,Libya,2018


Now we make four separate dataframes for each crop, because we'll use them to make four separate charts.

(In a perfect world, I would set up a loop for the last part of this analysis but... I'm not going to for just four charts.)

In [31]:
wheat = data[["Entity", "Wheat imports from Ukraine (% imports)"]].sort_values("Wheat imports from Ukraine (% imports)", ascending=False).dropna().head(10)
maize = data[["Entity", "Maize imports from Ukraine (% imports)"]].sort_values("Maize imports from Ukraine (% imports)", ascending=False).dropna().head(10)
barley = data[["Entity", "Barley imports from Ukraine (% imports)"]].sort_values("Barley imports from Ukraine (% imports)", ascending=False).dropna().head(10)
sunflower = data[["Entity", "Sunflower oil imports from Ukraine (% imports)"]].sort_values("Sunflower oil imports from Ukraine (% imports)", ascending=False).dropna().head(10)

In [32]:
display(wheat)

,Entity,Wheat imports from Ukraine (% imports)
925,Lebanon,66.726590
1683,Tunisia,53.174998
954,Libya,43.341057
604,Gambia,41.350635
1364,Russia,32.278183
1643,Thailand,31.973444
805,Israel,28.137004
765,Indonesia,24.963575
1055,Mauritania,24.216582
434,Democratic Republic of Congo,23.520446


Now let's define the deck, which will have some HTML buttons that allow users to switch between the different crops.

In [33]:
deck = f'Top 10 countries by percentage of imported crop from Ukraine in 2019.<br><br><b>Crop</b>: <span style="line-height:30px"><a target="_self" href="https://datawrapper.dwcdn.net/ZVNmh/" style="background:#313695; padding:1px 6px; border-radius:5px; color:#ffffff; font-weight:400;cursor:pointer;" rel="nofollow noopener noreferrer">Wheat</a> &nbsp;<a target="_self" href="https://datawrapper.dwcdn.net/gqB57/" style="background:#ABD9E9; padding:1px 6px; border-radius:5px; color:#ffffff; font-weight:400;cursor:pointer;" rel="nofollow noopener noreferrer">Maize</a> &nbsp;<a target="_self" href="https://datawrapper.dwcdn.net/16z9B/" style="background:#FDAE61; padding:1px 6px; border-radius:5px; color:#ffffff; font-weight:400;cursor:pointer;" rel="nofollow noopener noreferrer">Barley</a> &nbsp;<a target="_self" href="https://datawrapper.dwcdn.net/4zIw2/" style="background:#A50026; padding:1px 6px; border-radius:5px; color:#ffffff; font-weight:400;cursor:pointer;" rel="nofollow noopener noreferrer">Sunflower oil</a> &nbsp;</span>'

Now we implement datawrappergraphics's Chart() class, which updates the chart (I made the charts ahead of time using Datawrapper's GUI). The we display the final product.

In [34]:
(datawrappergraphics.Chart("ZVNmh")
    .data(wheat)
    .head(f"Percentage of imported wheat coming from Ukraine")
    .deck(deck)
    .publish()
)

(datawrappergraphics.Chart("gqB57")
    .data(maize)
    .head(f"Percentage of imported maize coming from Ukraine")
    .deck(deck)
    .publish()
)

(datawrappergraphics.Chart("16z9B")
    .data(barley)
    .head(f"Percentage of imported barley coming from Ukraine")
    .deck(deck)
    .publish()
)

(datawrappergraphics.Chart("4zIw2")
    .data(sunflower)
    .head(f"Percentage of imported sunflower oil coming from Ukraine")
    .deck(deck)
    .publish()
    .show()
)

c:\Users\dexmc\anaconda3\envs\geo\lib\site-packages\IPython\core\display.py:419: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


That's all!

\-30\-